# **Cleaning & Normalization**

#### _We import the libraries we will use and additional configuration_

In [9]:
import pandas as pd
import numpy as np
import os

# --- Configuration ---
INPUT_FILE = "ventas.csv"
OUTPUT_FOLDER = "cleaned_seeders"


## **_1. Setup and file loading_**

In [10]:
print(f"--- Starting Data Processing for: {INPUT_FILE} ---")

try:
    # Create output folder if it doesn't exist
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print(f"Output Folder '{OUTPUT_FOLDER}' created.")
        
    print(f"Loading file: {INPUT_FILE}")
    df = pd.read_csv(INPUT_FILE)
    
    # Clean column names (replace spaces with underscores)
    df.columns = [col.replace(' ', '_') for col in df.columns]
    
    print("File loaded and column names cleaned.")
    
except FileNotFoundError:
    print(f"Error: The input file '{INPUT_FILE}' was not found. Please check the path.")
    # Exit if the file is not found
    raise 
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    # Exit on other critical errors
    raise


--- Starting Data Processing for: ventas.csv ---
Output Folder 'cleaned_seeders' created.
Loading file: ventas.csv
File loaded and column names cleaned.


## **_2. Data cleaning_**

In [11]:
try:
    print("Starting Phase 1: Data Cleaning...")

    # 1. Remove duplicates
    rows_before = len(df)
    df.drop_duplicates(inplace=True)
    print(f"  Removed {rows_before - len(df)} duplicate rows.")

    # 2. Clean Text (Categorical) Columns
    string_columns = ['Ciudad', 'Producto', 'Tipo_Producto', 'Tipo_Venta', 'Tipo_Cliente']
    
    for col in string_columns:
        # Normalization (Uppercase, strip spaces, remove accents/special chars)
        df[col] = df[col].astype(str).str.upper().str.strip()
        df[col] = df[col].str.replace('Á', 'A').str.replace('É', 'E').str.replace('Í', 'I')
        df[col] = df[col].str.replace('Ó', 'O').str.replace('Ú', 'U').str.replace('[^A-Z0-9_ ]', '', regex=True)
    print("  Categorical columns cleaned and normalized.")

    # 3. Clean Numeric Columns
    numeric_columns = ['Cantidad', 'Precio_Unitario', 'Descuento', 'Costo_Envio', 'Total']

    for col in numeric_columns:
        # Handle decimal separators (comma to dot) and convert to numeric
        df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce') # Coerce non-numeric to NaN
        
        # Fill NaN with 0
        df[col].fillna(0, inplace=True)
        
        # Handle negative values for Quantity/Unit Price
        if col in ['Cantidad', 'Precio_Unitario']:
             df.loc[df[col] < 0, col] = df[col].abs()
             
    print("  Numeric columns cleaned and standardized.")

    # 4. Clean Date Column
    df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')
    df['Fecha'].fillna(pd.to_datetime('1900-01-01'), inplace=True) # Fill missing dates
    print("  Date column validated and missing values filled.")

    print("Phase 1: Data Cleaning completed successfully.")
    
except Exception as e:
    print(f"An error occurred during the Data Cleaning phase: {e}")



Starting Phase 1: Data Cleaning...
  Removed 4068 duplicate rows.
  Categorical columns cleaned and normalized.


C:\Users\alber\AppData\Local\Temp\ipykernel_21588\1313898549.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(0, inplace=True)
C:\Users\alber\AppData\Local\Temp\ipykernel_21588\1313898549.py:28: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doin

  Numeric columns cleaned and standardized.
  Date column validated and missing values filled.
Phase 1: Data Cleaning completed successfully.


C:\Users\alber\AppData\Local\Temp\ipykernel_21588\1313898549.py:38: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Fecha'].fillna(pd.to_datetime('1900-01-01'), inplace=True) # Fill missing dates


## **_3. Normalization and seeder generation_**

In [13]:
try:
    print("Starting Phase 2: Normalization and Seeder Generation...")

    # Define dimension tables (Keys: Table Name, Values: List of Columns)
    dimension_tables = {
        "City": ["Ciudad"],
        "Product": ["Producto"], 
        "Tipo_Venta": ["Tipo_Venta"],
        "Tipo_Cliente": ["Tipo_Cliente"]
    }

    dimension_dfs = {}
    join_columns = []

    # 3.1. Generate Dimension Tables (Dim)
    for table, columns in dimension_tables.items():
        table_name = f"Dim{table}"
        id_col = f"ID_{table}" 
        
        print(f"  Generating seeder: {table_name}")
        
        # Create dimension table (unique values + ID)
        df_dim = df[columns].drop_duplicates().reset_index(drop=True)
        df_dim.insert(0, id_col, df_dim.index + 1)
        
        # Save Dimension Table
        output_filename = os.path.join(OUTPUT_FOLDER, f"seeder_{table_name}.csv")
        df_dim.to_csv(output_filename, index=False)
        
        dimension_dfs[table] = df_dim
        join_columns.extend(columns) 

    # 3.2. Generate Sales Fact Table (FactSales)
    print("\n  Generating seeder: FactSales")

    # Metrics and non-dimension attributes for the Fact table
    sales_attributes = [
        'Fecha', 'Tipo_Producto', 'Cantidad', 'Precio_Unitario', 
        'Descuento', 'Costo_Envio', 'Total'
    ]
    
    # Start FactSales DF with attributes and original join columns
    initial_columns = sales_attributes + join_columns
    df_sales = df[initial_columns].copy()
    
    final_sales_columns = sales_attributes.copy()

    # Merge Foreign Keys
    for table, columns in dimension_tables.items():
        df_dim = dimension_dfs[table]
        fk_col = df_dim.columns[0] 

        # Left merge to get the Foreign Key ID
        df_sales = pd.merge(
            df_sales, 
            df_dim, 
            on=columns, 
            how='left'
        )
        
        final_sales_columns.insert(0, fk_col)

    # Finalize FactSales DF
    df_sales = df_sales.drop(columns=join_columns, errors='ignore')
    
    # Insert Primary Key (ID_Venta)
    df_sales.insert(0, 'ID_Venta', df_sales.reset_index(drop=True).index + 1)
    final_sales_columns.insert(0, 'ID_Venta')
    
    # Select final columns in correct order
    df_sales = df_sales[final_sales_columns].copy()

    # Save the FactSales table
    output_filename = os.path.join(OUTPUT_FOLDER, f"seeder_FactSales.csv")
    df_sales.to_csv(output_filename, index=False)
    print(f"  Saved: {output_filename} with {len(df_sales)} rows.")
    
    print("\nPhase 2: Normalization and Seeder Generation completed successfully.")

except Exception as e:
    print(f"An error occurred during the Normalization phase: {e}")


Starting Phase 2: Normalization and Seeder Generation...
  Generating seeder: DimCity
  Generating seeder: DimProduct
  Generating seeder: DimTipo_Venta
  Generating seeder: DimTipo_Cliente

  Generating seeder: FactSales
  Saved: cleaned_seeders\seeder_FactSales.csv with 1245932 rows.

Phase 2: Normalization and Seeder Generation completed successfully.
